# Ejercicio 1 Parcial 3 - Métodos Númericos

__Juan David Parra Cantor__

*__ID:__* 000324320

__Ejercicio Propuesto__

![Planteamiento del Problema](ejercicio1.jpg)

## Pasos Generales para Resolver el Problema

1. **Ecuación Diferencial**: El problema es una viga sometida a una carga distribuida, y la ecuación diferencial que modela el comportamiento de la viga es:

   $$
   \frac{d^2y}{dx^2} = \frac{wLx^2}{2EI} - \frac{wx^2}{2EI}
   $$

   Con las condiciones de frontera:

   $$
   y(0) = y(L) = 0
   $$

2. **Método del Disparo Lineal**: Este método convierte el problema de valores en la frontera en un problema de valores iniciales. Supongamos que conocemos \( y'(0) \), luego resolvemos el sistema de ecuaciones diferenciales. Ajustamos \( y'(0) \) hasta que se satisfaga la condición \( y(L) = 0 \).

3. **Método de Diferencias Finitas**: Este método discretiza la ecuación diferencial en una malla de puntos. Al hacerlo, se obtiene un sistema de ecuaciones lineales que se puede resolver numéricamente.

4. **Solución Exacta**: Como se muestra en la imagen, la solución exacta ya está dada:

   $$
   y(x) = \frac{wLx^3}{12EI} - \frac{wx^4}{24EI} - \frac{wL^3x}{24EI}
   $$




## Videos

A continuación se presentan los videos de explicación del código contenido en este cuaderno de Jupyter

__Parte 1:__ https://www.youtube.com/watch?v=3X2SsFX5kHE

__Parte 2:__ https://youtu.be/uiSWYy__7Aw

## Solución Paso a paso

## 1. Constantes del Problema

Se definen las constantes físicas del problema: módulo de elasticidad \( E \), momento de inercia \( I \), carga distribuida \( w \) y la longitud de la viga \( L \). Todas las unidades se convierten a sistema de pulgadas para que sean consistentes con los cálculos.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.linalg import solve

# Parámetros del problema
# 1. Módulo de elasticidad (Ksi a psi)
E_ksi = 30  # Módulo de elasticidad en Ksi
E = E_ksi * 1000  # Conversión de Ksi a psi

# 2. Momento de inercia (in^4)
I = 800  # Momento de inercia en in^4 (sin conversión necesaria)

# 3. Carga distribuida (kip/ft a lb/in)
w_kip_ft = 1  # Carga distribuida en kip/ft
w = w_kip_ft * 1000 / 12  # Conversión de kip/ft a lb/in

# 4. Longitud de la viga (ft a in)
L_ft = 10  # Longitud de la viga en pies
L = L_ft * 12  # Conversión de pies a pulgadas

# Resultados
print(f"Módulo de elasticidad E: {E} psi")
print(f"Momento de inercia I: {I} in^4")
print(f"Carga distribuida w: {w} lb/in")
print(f"Longitud de la viga L: {L} in")


## 2. Solución Exacta

Implementamos la ecuación exacta que describe la deflexión de la viga. Esta ecuación está basada en la fórmula proporcionada en el enunciado:

$$
y(x) = \frac{wLx^3}{12EI} - \frac{wx^4}{24EI} - \frac{wL^3x}{24EI}
$$

Esta ecuación será evaluada para diferentes valores de \( x \) a lo largo de la viga.

In [ ]:
# Nueva solución exacta (según la imagen)
def solucion_exacta(x):
    return (w * L * x**3) / (12 * E * I) - (w * x**4) / (24 * E * I) - (w * L**3 * x) / (24 * E * I)

x_exacta = np.linspace(0, L, 100)
y_exacta = solucion_exacta(x_exacta)

# Graficar todas las soluciones
plt.figure(figsize=(10, 6))
plt.plot(x_exacta, y_exacta, label="Solución Exacta", color="black", linewidth=2)
plt.xlabel("x (in)")
plt.ylabel("y (in)")
plt.title("Solución exacta")
plt.legend()
plt.grid(True)
plt.show()

## 3. Método del Disparo Lineal

Este método transforma un problema de frontera en un problema de valores iniciales. Se resuelve la ecuación diferencial 

$$
y''(x) = \text{fuerza}
$$

con una condición inicial \( y(0) = 0 \), y se ajusta el valor de \( y'(0) \) hasta que la condición \( y(L) = 0 \) se cumpla. Esto se hace usando un método de bisección.

In [ ]:
def disparo_lineal():
    # Definir el sistema de ecuaciones diferenciales
    def sistema_ecuaciones(x, y):
        # y[0] es la función desconocida (desplazamiento) y y[1] es su derivada (velocidad)
        # La ecuación diferencial es: y'' = (w * L * x / (2 * E * I)) - (w * x**2 / (2 * E * I))
        return [y[1], (w * L * x / (2 * E * I)) - (w * x**2 / (2 * E * I))]
    
    # Encontrar la condición inicial óptima mediante bisección
    a, b = -1, 1  # Intervalo inicial para buscar la condición inicial óptima
    tol = 1e-6  # Tolerancia para detener la búsqueda
    while b - a > tol:
        y_prime_0 = (a + b) / 2  # Punto medio del intervalo actual
        sol = solve_ivp(sistema_ecuaciones, [0, L], [0, y_prime_0], t_eval=np.linspace(0, L, 100))
        # Verificar si la solución encontrada satisface la condición de frontera en x=L
        if sol.y[0][-1] > 0:
            b = y_prime_0  # Si la condición no se cumple, reducir el intervalo hacia abajo
        else:
            a = y_prime_0  # Si la condición se cumple, reducir el intervalo hacia arriba
    
    # Resolver con la condición inicial óptima encontrada
    sol = solve_ivp(sistema_ecuaciones, [0, L], [0, y_prime_0], t_eval=np.linspace(0, L, 100))
    return sol.t, sol.y[0]  # Devolver la solución encontrada

# Método del disparo lineal
x_disparo, y_disparo = disparo_lineal()

# Graficar todas las soluciones
plt.figure(figsize=(10, 6))
plt.plot(x_disparo, y_disparo, label="Método del Disparo Lineal", linestyle="--", color="blue")
plt.xlabel("x (in)")
plt.ylabel("y (in)")
plt.title("Disparo Lineal")
plt.legend()
plt.grid(True)
plt.show()


## 4. Método de Diferencias Finitas

El método de diferencias finitas discretiza la ecuación diferencial utilizando una malla de puntos a lo largo de la viga. Luego, se construye un sistema de ecuaciones lineales que se resuelve usando álgebra lineal numérica.


In [ ]:
# Método de diferencias finitas
def diferencias_finitas(n_puntos):
    # Calcular el paso de discretización (dx)
    dx = L / (n_puntos + 1)
    
    # Crear un arreglo de puntos x con n_puntos + 2 elementos (incluyendo los extremos)
    x = np.linspace(0, L, n_puntos + 2)
    
    # Crear una matriz A y un vector b para el sistema de ecuaciones lineales
    A = np.zeros((n_puntos, n_puntos))
    b = np.zeros(n_puntos)
    
    # Rellenar la matriz A y el vector b
    for i in range(n_puntos):
        xi = (i + 1) * dx  # Calcular el valor de x en el punto i
        A[i, i] = -2 / dx**2  # Elemento diagonal de la matriz A
        if i > 0:
            A[i, i-1] = 1 / dx**2  # Elemento subdiagonal de la matriz A
        if i < n_puntos - 1:
            A[i, i+1] = 1 / dx**2  # Elemento superdiagonal de la matriz A
        b[i] = (w * L * xi / (2 * E * I)) - (w * xi**2 / (2 * E * I))  # Elemento del vector b
    
    # Resolver el sistema de ecuaciones lineales
    y_finitas = solve(A, b)
    
    # Agregar los valores de las condiciones de frontera (y(0) = y(L) = 0)
    y_finitas = np.concatenate(([0], y_finitas, [0]))
    
    return x, y_finitas  # Devolver la solución encontrada

# Método de diferencias finitas (con 50 puntos)
x_finitas, y_finitas = diferencias_finitas(50)

# Graficar todas las soluciones
plt.figure(figsize=(10, 6))
plt.plot(x_finitas, y_finitas, label="Método de Diferencias Finitas", linestyle=":", color="red")
plt.xlabel("x (in)")
plt.ylabel("y (in)")
plt.title("Diferencias Finitas")
plt.legend()
plt.grid(True)
plt.show()

## 5. Gráfico Comparativo

Finalmente, se grafican las tres soluciones (exacta, disparo lineal y diferencias finitas) para comparar visualmente los resultados de los diferentes métodos.

In [ ]:
# Graficar los resultados
x_exacta = np.linspace(0, L, 100)
y_exacta = solucion_exacta(x_exacta)

# Método del disparo lineal
x_disparo, y_disparo = disparo_lineal()

# Método de diferencias finitas (con 50 puntos)
x_finitas, y_finitas = diferencias_finitas(50)

# Graficar todas las soluciones
plt.figure(figsize=(10, 6))
plt.plot(x_exacta, y_exacta, label="Solución Exacta", color="black", linewidth=2)
plt.plot(x_disparo, y_disparo, label="Método del Disparo Lineal", linestyle="--", color="blue")
plt.plot(x_finitas, y_finitas, label="Método de Diferencias Finitas", linestyle=":", color="red")
plt.xlabel("x (in)")
plt.ylabel("y (in)")
plt.title("Comparación de Métodos para la Deflexión de la Viga")
plt.legend()
plt.grid(True)
plt.show()